# 实验1：分三轮对话分步提取四项指标（全局会话记忆）

本 Notebook 用于：
- 对一批论文（Title/Abstract/Keywords）进行结构化标注：
  1) 是否城市更新研究（1/0）
  2) 空间研究/非空间研究（1/0）
  3) 空间等级（国家/省级（州）/市级/城市群/）
  4) 具体空间描述（国家-省州-市-具体城市）
- 分三轮对话逐步提取（一步一个提示词一个结果），并在不超上下文上限前保持全局单会话记忆。
- 对比 DeepSeek V3.2 非思考（deepseek-chat）与思考（deepseek-reasoner）的效果。


## 0. 准备：填写 scripts/.env

请确保 `scripts/.env` 文件包含以下配置（已自动修复，请勿随意修改键名）：

```
CHAT_MODEL=deepseek-chat
CHAT_BASE_URL=https://api.deepseek.com/v1/chat/completions
CHAT_API_KEY_ENV=DEEPSEEK_API_KEY

REASONER_MODEL=deepseek-reasoner
REASONER_BASE_URL=https://api.deepseek.com/v1/chat/completions
REASONER_API_KEY_ENV=DEEPSEEK_API_KEY

DEEPSEEK_API_KEY=sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
```


In [1]:
from pathlib import Path
import json
import pandas as pd
import runpy
import os

def find_project_root():
    current = Path.cwd()
    # 向上寻找包含 data 和 scripts 目录的文件夹
    for parent in [current] + list(current.parents):
        if (parent / "data").exists() and (parent / "scripts").exists():
            return parent
    return current

PROJECT_ROOT = find_project_root()
print(f"Project Root: {PROJECT_ROOT}")

ENV_PATH = PROJECT_ROOT / "scripts" / ".env"
print(f"Env Path: {ENV_PATH}")

Project Root: C:\Users\26409\Desktop\城市更新论文
Env Path: C:\Users\26409\Desktop\城市更新论文\scripts\.env


In [2]:
print("ENV exists:", ENV_PATH.exists())
if ENV_PATH.exists():
    print(ENV_PATH.read_text(encoding="utf-8")[:500])

ENV exists: True
# =========================
# LLM 抽取实验配置（推荐）
# =========================
# Experiment1（分三轮对话）：
CHAT_MODEL=deepseek-chat
CHAT_BASE_URL=https://api.deepseek.com/v1/chat/completions
CHAT_API_KEY_ENV=DEEPSEEK_API_KEY

REASONER_MODEL=deepseek-reasoner
REASONER_BASE_URL=https://api.deepseek.com/v1/chat/completions
REASONER_API_KEY_ENV=DEEPSEEK_API_KEY

DEEPSEEK_API_KEY=sk-edb354d84503468ab826821cc57e53a0

# StudyArea/其它任务（如需）：
# PRIMARY_MODEL=你的模型名称
# PRIMARY_BASE_URL=你的API端点URL
# PRIMARY_API_KEY_ENV=


## 1. 加载实验代码（从现有脚本中载入函数）

这里使用 `runpy.run_path` 载入 `extract_study_area_llm.py`，不会执行 main（脚本有 `if __name__ == '__main__'` 保护）。


In [3]:
EXTRACT_SCRIPT = PROJECT_ROOT / "scripts" / "llm_extraction" / "python" / "extract_study_area_llm.py"
ns = runpy.run_path(str(EXTRACT_SCRIPT))

_load_env_from_file = ns["_load_env_from_file"]
ModelConfig = ns["ModelConfig"]
run_experiment1_batch = ns["run_experiment1_batch"]

_load_env_from_file(str(ENV_PATH))
print("Loaded.")

Loaded.


## 2. 选择输入数据

- 默认示例输入：`data/raw/Urban Renovations_Article_V2.0.xlsx`
- 如需换数据，只改 `input_xlsx` 和 `sheet_name`。


In [4]:
input_xlsx = str(PROJECT_ROOT / "data" / "raw" / "Urban Renovations_Article_V2.0.xlsx")
sheet_name = 0

title_col = "Article Title"
abstract_col = "Abstract"
keywords_col = "Author Keywords"

pd.ExcelFile(input_xlsx, engine="openpyxl").sheet_names

['Sheet1']

## 3. 运行参数（shot / 对比模式 / 上下文预算 / trace）

- `shot`：`zero` / `one` / `few`
- `compare_models=True`：同时跑 deepseek-chat 与 deepseek-reasoner（两套全局会话记忆）
- `context_char_budget`：字符级保守预算（超过才允许压缩/重启，并在输出列里标记）
- `trace_jsonl`：保存每一步调用的 prompt、raw 输出、messages（full/tail/none）


In [5]:
shot = "few"  # "zero" | "one" | "few"
compare_models = True

max_rows = 30
resume = False

# DeepSeek Reasoner 需要较长的输出 token（包含思考过程）
# 建议至少 4096 或 8192
max_tokens = 8192
temperature = 0.1
timeout_s = 120

context_char_budget = 250_000

trace_jsonl = str(PROJECT_ROOT / "data" / "processed" / "experiment1_trace.jsonl")
trace_level = "tail"  # "full" | "tail" | "none"
trace_tail_messages = 50

debug_columns = True

output_csv = str(PROJECT_ROOT / "data" / "processed" / "experiment1_output.csv")
output_xlsx = str(PROJECT_ROOT / "data" / "processed" / "experiment1_output.xlsx")

print(output_xlsx)

C:\Users\26409\Desktop\城市更新论文\data\processed\experiment1_output.xlsx


## 4. 配置模型（DeepSeek 非思考/思考）

模型与 API 端点统一从 `scripts/.env` 读取（便于实验控制与复现）。

建议在 `scripts/.env` 中配置：
```
CHAT_MODEL=deepseek-chat
CHAT_BASE_URL=https://api.deepseek.com/v1/chat/completions
CHAT_API_KEY_ENV=DEEPSEEK_API_KEY

REASONER_MODEL=deepseek-reasoner
REASONER_BASE_URL=https://api.deepseek.com/v1/chat/completions
REASONER_API_KEY_ENV=DEEPSEEK_API_KEY

DEEPSEEK_API_KEY=你的key
```


In [6]:
import os

chat_cfg = ModelConfig(
    provider="deepseek",
    model=os.environ["CHAT_MODEL"],
    api_key_env=os.environ.get("CHAT_API_KEY_ENV", "DEEPSEEK_API_KEY"),
    base_url=os.environ["CHAT_BASE_URL"],
)

reasoner_cfg = ModelConfig(
    provider="deepseek",
    model=os.environ["REASONER_MODEL"],
    api_key_env=os.environ.get("REASONER_API_KEY_ENV", "DEEPSEEK_API_KEY"),
    base_url=os.environ["REASONER_BASE_URL"],
)

chat_cfg, reasoner_cfg

(ModelConfig(provider='deepseek', model='deepseek-chat', api_key_env='DEEPSEEK_API_KEY', base_url='https://api.deepseek.com/v1/chat/completions'),
 ModelConfig(provider='deepseek', model='deepseek-reasoner', api_key_env='DEEPSEEK_API_KEY', base_url='https://api.deepseek.com/v1/chat/completions'))

## 5. Dry-run（可选）：不调用模型，仅验证输出列/trace结构

如果你想先确认“分三步写回 Excel 的列是否正确”，可以先跑 dry-run。


In [7]:
dry_run = False

run_experiment1_batch(
    input_xlsx=input_xlsx,
    sheet_name=sheet_name,
    output_csv=output_csv,
    output_xlsx=output_xlsx,
    chat_cfg=chat_cfg,
    reasoner_cfg=reasoner_cfg if compare_models else None,
    compare_models=compare_models,
    shot=shot,
    max_rows=max_rows,
    resume=resume,
    title_col=title_col,
    abstract_col=abstract_col,
    keywords_col=keywords_col,
    max_tokens=max_tokens,
    temperature=temperature,
    timeout_s=timeout_s,
    context_char_budget=context_char_budget,
    trace_jsonl=trace_jsonl,
    trace_level=trace_level,
    trace_tail_messages=trace_tail_messages,
    debug_columns=debug_columns,
    dry_run=dry_run,
)

Experiment1 标注:   0%|          | 0/30 [00:00<?, ?条/s]


[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=2591


Experiment1 标注:  10%|█         | 3/30 [04:21<39:13, 87.18s/条]


[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=2086

[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=2375

[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=1926


Experiment1 标注:  13%|█▎        | 4/30 [05:49<37:50, 87.33s/条]

...


[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=1790

[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=1548

[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=1495


Experiment1 标注:  97%|█████████▋| 29/30 [44:12<01:43, 103.66s/条]


[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=1677

[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=1462

[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=2703


Experiment1 标注: 100%|██████████| 30/30 [45:46<00:00, 91.56s/条] 


[Error] Session 0 failed: api_error:RuntimeError:Empty content from deepseek (deepseek-reasoner). Keys: ['role', 'content', 'reasoning_content'], reasoning_len=1783


## 6. 查看输出 Excel（四项指标 + 调试列）


In [ ]:
out_df = pd.read_excel(output_xlsx, engine="openpyxl")
out_df.columns[:30], len(out_df)

In [ ]:
if compare_models:
    cols = [
        "chat_UR_IsUrbanRenewal",
        "chat_Spatial_IsSpatial",
        "chat_Spatial_Level",
        "chat_Spatial_Location",
        "reasoner_UR_IsUrbanRenewal",
        "reasoner_Spatial_IsSpatial",
        "reasoner_Spatial_Level",
        "reasoner_Spatial_Location",
    ]
else:
    cols = ["UR_IsUrbanRenewal", "Spatial_IsSpatial", "Spatial_Level", "Spatial_Location"]
out_df[cols].head(10)

## 7. 查看 trace（观察上下文与是否发生重启）

- `context_reset=1` 表示发生了上下文压缩/重启。
- `messages` 字段里可看到当次调用保存的上下文（tail 或 full）。


In [ ]:
trace_path = Path(trace_jsonl)
print("trace exists:", trace_path.exists(), "size:", trace_path.stat().st_size if trace_path.exists() else None)
if trace_path.exists():
    lines = trace_path.read_text(encoding="utf-8").splitlines()
    print("trace lines:", len(lines))
    last = json.loads(lines[-1])
    {k: last.get(k) for k in ["session_id","paper_key","round","model","context_chars_before","context_chars_after","context_reset","context_reset_reason","parsed_ok","repairs"]}